# Entrega

### Imports

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from joblib import dump, load

In [2]:
# from sklearn.model_selection import StratifiedKFold

In [3]:
from preprocessing import reemplazarNulls,reemplazarCategoricas,reemplazarFechas,regularizar,targetBooleano

[###] All Done                                              


In [20]:
from utilities import score2, score

### Test Holdout

In [5]:
df_feat = pd.read_csv("datasets/holdout_features.csv", low_memory=False)
df_targ = pd.read_csv("datasets/holdout_target.csv").llovieron_hamburguesas_al_dia_siguiente

In [6]:
df_targ=targetBooleano(df_targ)
reemplazarNulls(df_feat , inplace=True)
reemplazarCategoricas(df_feat , inplace=True)
reemplazarFechas(df_feat , inplace=True)
regularizar(df_feat , inplace=True)

,id,direccion_viento_tarde,direccion_viento_temprano,horas_de_sol,humedad_tarde,humedad_temprano,mm_lluvia_dia,nubosidad_tarde,presion_atmosferica_tarde,rafaga_viento_max_direccion,rafaga_viento_max_velocidad,velocidad_viendo_tarde
0,-0.114786,-0.311222,-0.773796,-2.018030,-1.372610,-0.832696,0.345041,0.916539,-1.384970,-0.220589,1.544505,0.834079
1,1.006900,-1.358982,-0.976624,-2.018030,1.085157,0.638263,-0.281713,0.916539,-2.080499,0.802215,0.221665,-0.642648
2,0.008030,-1.149430,-0.570969,-2.018030,-0.023248,1.058537,1.020008,0.916539,-0.064885,-0.016028,-0.219282,-0.869837
3,0.337019,-0.939878,-0.976624,0.653494,-0.360588,-0.044682,-0.281713,0.916539,-0.717831,1.620459,-0.219282,0.152513
4,1.129419,-1.149430,-1.179451,-2.018030,0.362285,0.165455,-0.281713,0.181553,0.559671,-1.038833,-0.219282,0.379701
...,...,...,...,...,...,...,...,...,...,...,...,...
11368,0.913813,-0.101670,-0.773796,-1.515466,0.217710,0.270523,-0.257607,0.916539,0.971310,-0.834272,-0.513247,-0.188270
11369,0.358833,-1.568534,0.645996,-2.018030,-0.264205,-0.517490,-0.281713,-1.655914,-0.121663,-0.220589,-0.660229,-1.097025
11370,-1.586621,-0.101670,-0.570969,0.045127,-1.468993,-1.515641,-0.281713,0.916539,-1.015914,0.393094,3.014328,3.333155
11371,-1.313430,1.574745,1.254478,-2.018030,1.085157,0.795865,-0.209395,0.916539,1.553283,1.211337,0.221665,-0.415459


In [7]:
# skf = StratifiedKFold(n_splits=5, random_state=1, shuffle=False)
# skf.get_n_splits(df_feat, df_targ)

### Modelos

In [8]:
arbol = load('models/Tree/arbol_r.sk')

In [9]:
knn = load('models/KNN/knn_auto.sk')

In [10]:
nb = load('models/NB/nb_reg.sk')

In [11]:
svm_lin = load('models/SVM/svm_lineal.sk')
svm_pol = load('models/SVM/svm_poly.sk')
svm_rad = load('models/SVM/svm_radial.sk')

In [12]:
nn = load('models/NN/mlp_cls_r.sk')

In [13]:
rand_f = load('models/Ensambles/forest_r.sk')

In [14]:
boost = load('models/Ensambles/boost_r.sk')

In [15]:
# vote = load('models/Ensambles/vote_svm.sk')

## Comparacion con Test Holdout

In [16]:
modelos = [
    ('arbol',arbol),
    ('knn',knn),
    ('nb',nb),
    ('svm_lin',svm_lin),
    ('svm_pol',svm_pol),
    ('svm_rad',svm_rad),
    ('nn',nn),
    ('rand_f',rand_f),
    ('boost',boost),
]

In [17]:
df = pd.DataFrame(columns=["Modelo", "Preprocesamientos", "AUC-ROC", "Accuracy", "Precision", "Recall", "F1 score"])

In [22]:
pred = arbol.predict(df_feat)
prob = arbol.predict_proba(df_feat)
score(df_targ, pred, prob[:,1])

ValueError: unknown format is not supported

In [ ]:
for (name, model) in modelos:
    pred = model.predict(df_feat)
    prob = model.predict_proba(df_feat)
    
    roc_auc, acc, prec, rec, f1 = score2(df_targ, pred, prob)
    
    df = df.append(
        {
            "Modelo": name,
            "Preprocesamientos":  "Todos",
            "AUC-ROC":  roc_auc,
            "Accuracy":  acc,
            "Precision":  prec,
            "Recall":  rec,
            "F1 score":  f1,
        },
        ignore_index=True
    )
df